In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthBegin, DateOffset
import numpy as np
import calendar
import os

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf
import repofuncs.reapmonthlyfuncs as rmf

import warnings
warnings.filterwarnings('ignore')

path_to_matrix = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall"
path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)
mf.read_directory()

report_data = pd.read_csv('Netcall Dashboard Data.csv',low_memory=False)
report_data = mf.strip_dataframe(report_data)

lookup_data = pd.read_excel('Queue to Group Lookup.xlsx')
lookup_data = mf.strip_dataframe(lookup_data)

Your Current Directory is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data
DataFrame: files
Number of records: 13 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,Interactions Apr 2025 (excl Cust Serv)
1,Interactions Feb 2023 - Apr 2024 (excl Cust Serv)
2,Interactions Feb 2025 (excl Cust Serv)
3,Interactions Jan 2022 - Dec 2022 (excl Cust Serv)
4,Interactions Jan 2024 - Jun 2024 (excl Cust Serv)
5,Interactions July 2025 (excl Cust Serv)
6,Interactions Jun 24 - Jan 25 (excl Cust Serv)
7,Interactions Mar 2025 (excl Cust Serv)
8,Interactions May 2025 - Jun 2025 (excl Cust Serv)
9,Interactions_Jan 2023 (LIMITED GROUPS)


In [2]:
mf.validate_data(report_data)

#####################################################################################################################################################################
Validation Dataframe
Number of records: 225,612 | Number of fields: 13
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
0,Inbound,Call,2022-01-04,ACE and Admissions,ACE and Admissions,Answered,15,4241.0,5580.0,5580.0,5349.0,0.0,9590.0
1,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Abandoned,2,0.0,240.0,240.0,0.0,0.0,0.0
2,Inbound,Call,2022-01-04,Adult Education,General Enquiry,Answered,66,1954.0,25260.0,25260.0,23670.0,0.0,25624.0
3,Inbound,Call,2022-01-04,CAS Team 1 6945,CAS Team 1 6945 Queue,Answered,3,180.0,300.0,300.0,210.0,0.0,390.0
4,Inbound,Call,2022-01-04,CAS Team 2 6946,CAS Team 2 6946 Queue,Answered,3,180.0,360.0,360.0,295.0,0.0,475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225607,Outbound,Email,2025-08-14,Customer Services,Cost of Living Crisis Email,Sent,2,120.0,362.0,362.0,361.0,0.0,481.0
225608,Outbound,Email,2025-08-18,Customer Services,Cost of Living Crisis Email,Sent,2,66.0,364.0,364.0,363.0,0.0,429.0
225609,Outbound,Email,2025-08-20,Customer Services,Cost of Living Crisis Email,Sent,2,26.0,100.0,100.0,97.0,2.0,123.0
225610,Outbound,Email,2025-08-21,Customer Services,Cost of Living Crisis Email,Deleted,1,13.0,12.0,12.0,11.0,1.0,24.0


Unique Values
DataFrame: unique_df


,direction,type,dateofqueue,groupname,queue,result,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
0,Inbound,Call,2022-01-01,ACE and Admissions,ACE and Admissions,Abandoned,1.0,0.0,-32.0,-32.0,-98142.0,0.0,0.0
1,Outbound,Email,2022-01-02,ASB UNIT,ASB UNIT,Answered,2.0,1.0,-8.0,-8.0,-90007.0,1.0,1.0
2,--,Messaging,2022-01-03,Accessible Transport,Accessible Transport,Callback,3.0,2.0,-7.0,-7.0,-84754.0,2.0,2.0
3,--,--,2022-01-04,Adult Education,Accessible Transport Blue Badge,Deleted,4.0,3.0,-6.0,-6.0,0.0,3.0,3.0
4,--,--,2022-01-05,Adult Social Care,Accessible Transport Callback,Discarded,5.0,4.0,-5.0,-5.0,1.0,4.0,4.0
5,--,--,2022-01-06,CAS Team 1 6945,Accessible Transport Email,Failed,6.0,5.0,-4.0,-4.0,2.0,5.0,5.0
6,--,--,2022-01-07,CAS Team 2 6946,Accessible Transport Freedom Passes,Other,7.0,6.0,-3.0,-3.0,3.0,6.0,6.0
7,--,--,2022-01-08,CAS Team 3 6947,Accessible Transport Taxi Cards,Pending,8.0,7.0,-2.0,-2.0,4.0,7.0,7.0
8,--,--,2022-01-09,CAS Team 4 6948,Adult Social Care,Redirected,9.0,8.0,-1.0,-1.0,5.0,8.0,8.0
9,--,--,2022-01-10,CSD Reablement 6613,Agent Transfers,Replied,10.0,9.0,0.0,0.0,6.0,9.0,9.0



DataFrame: info_df
Max number of non-null records: 225,612


,Field Name,No. of Unique Values,No. of Null Values,No. of Blank Values,Data Type
0,direction,2,--,--,object
1,type,3,--,--,object
2,dateofqueue,"1,321",--,--,object
3,groupname,50,--,--,object
4,queue,194,--,--,object
5,result,12,--,--,object
6,number_of_interactions,431,--,--,int64
7,wrap_upduration,"10,102",--,--,float64
8,queue_time,"45,761",--,--,float64
9,duration,"45,761",--,--,float64



Summary statistics:


,number_of_interactions,wrap_upduration,queue_time,duration,talk_time,hold_time,agent_handling_time
count,225612.000000,2.256120e+05,2.256120e+05,2.256120e+05,225612.000000,2.256120e+05,2.256120e+05
mean,12.131673,8.004984e+02,3.267638e+05,3.267638e+05,3085.260992,6.945582e+03,4.479749e+03
std,32.465983,8.890754e+03,4.022666e+06,4.022666e+06,8662.711520,4.767584e+05,1.779128e+04
min,1.000000,0.000000e+00,-3.200000e+01,-3.200000e+01,-98142.000000,0.000000e+00,0.000000e+00
25%,1.000000,0.000000e+00,2.750000e+02,2.750000e+02,0.000000,0.000000e+00,0.000000e+00
50%,3.000000,3.100000e+01,1.291000e+03,1.291000e+03,332.000000,0.000000e+00,4.840000e+02
75%,11.000000,2.920000e+02,6.772000e+03,6.772000e+03,2042.000000,1.060000e+02,2.699000e+03
max,4154.000000,2.313121e+06,6.078473e+08,6.078473e+08,310823.000000,2.212321e+08,2.334216e+06


End of data validation
#####################################################################################################################################################################


In [3]:
lookup_columns = {
    'Group':'group',
    'Group Name':'groupname',
    'Queue Summary':'queuesummary',
    'Queue':'queue'
}

groups_to_order = {
    "Contact Centre":0,
    "Accessible Transport":1,
    "Benefits":2,
    "Complaints":3,
    "Cost of Living":4,
    "Council Tax":5,
    "Electoral Services":6,
    "Enforcement and Licensing":7,
    "Other":8,
    "Parking":9,
    "Registrars":10,
    "Sports and Leisure":11,
    "Switchboard":12,
    "Waste":13,
    "Repair CSC":14,
    "Parking Services":15,
    "Rent Income Services":16,
    "Housing Advice":17,
    "Adult Social Care":18,
    "Childrens Social Care":19,
    "Other Groups":20
}

columns_to_rename = {
    "group":"Group",
    "number_of_interactions":"Total Calls",
    "agent_handling_time":"Total Handling Time (Days)",
    "abandoned":"% Abandoned",
    "answered":"% Answered",
    "callback":"% Callback",
    "redirected":"% Redirected",
    "transferred":"% Transferred",
    r"% change 12 month average":r"% Change 12 Month Average",
    r"% change same month last year":r"% Change Same Month Last Year",
    "average_handling_time":"Average Handling Time (Minutes)",
    "average_queue_time":"Average Queue Time (Minutes)"
}

columns_to_order = [
    "Group",
    "Total Calls",
    "% Change Same Month Last Year",
    "% Change 12 Month Average",
    "% Answered",
    "% Abandoned",
    "% Transferred",
    "% Callback",
    "% Redirected",
    "Total Handling Time (Days)",
    "Average Handling Time (Minutes)",
    "Average Queue Time (Minutes)"
]

In [4]:
latest_date = pd.to_datetime(report_data['dateofqueue']).max()
first_of_current_month = latest_date.replace(day=1)
last_full_month = first_of_current_month - MonthBegin(1)
same_month_last_year = last_full_month - DateOffset(years=1)
last_full_month = last_full_month.strftime('%Y-%m')
same_month_last_year = same_month_last_year.strftime('%Y-%m')
last_full_month_name = calendar.month_abbr[latest_date.month-1]

Aggregating Data

In [5]:
report_df = report_data.copy()
report_df['monthofqueue'] = pd.to_datetime(report_df['dateofqueue']).dt.strftime("%Y-%m")

report_df = report_df.groupby(['direction','type','monthofqueue','groupname','queue','result']).agg(
    number_of_interactions = ('number_of_interactions', 'sum'),
    wrap_upduration = ('wrap_upduration','sum'),
    queue_time = ('queue_time','sum'),
    duration = ('duration','sum'),
    talk_time = ('talk_time','sum'),
    hold_time = ('hold_time','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
).reset_index()

report_df = report_df[
    (report_df['direction'] == 'Inbound') &
    (report_df['type'] == 'Call')
]

Merging the LookUp Data

In [6]:
report_df['composite_key'] = report_df['groupname'].str.lower() + ":" + report_df['queue'].str.lower()
lookup_data['composite_key'] = lookup_data['Group Name'].str.lower() + ":" + lookup_data['Queue'].str.lower()

report_df = report_df.merge(
    lookup_data, how='left',
    on='composite_key'
)

report_df.drop(
    columns=['groupname','queue', 'composite_key','direction','type'],
    inplace=True
)

report_df.rename(
    columns=lookup_columns,
    inplace=True
)

Pivoting the Call Result Data

In [7]:
results_data = report_df.pivot_table(
    index=['monthofqueue','group','queuesummary'],
    columns='result',
    values=['number_of_interactions'],
    aggfunc='sum'
)

results_data.columns = results_data.columns.droplevel(0)

results_data.columns = [column.lower() for column in results_data.columns]

results_data['total'] = results_data[results_data.columns.to_list()].sum(axis=1)

Aggregating the Main Report Data then Merging the Pivoted Call Results back to the Main Dataset

In [8]:
report_df = report_df.groupby(['monthofqueue','group','queuesummary']).agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum')
)

report_df = report_df.merge(
    results_data, how='left',
    left_index=True, right_index=True
).reset_index().sort_values(by=['monthofqueue','group','queuesummary'],ascending=False)

report_df['check'] = report_df['number_of_interactions'] == report_df['total']

if False not in report_df['check'].unique():
    report_df.drop(
        columns=['total','check','failed','other'],
        inplace=True
    )
else:
    raise ValueError('The number of interactions between the call results data and the main report data are not aligned!')

report_df['month'] = pd.to_datetime(report_df['monthofqueue'],format='%Y-%m').dt.month

report_df.sort_values(
    by=['group','queuesummary','monthofqueue'],
    inplace=True
)

Transformations being applied to the Contact Centre Data

In [9]:
columns_to_cummulate = [
    'number_of_interactions',
    'agent_handling_time',
    'queue_time',
    'abandoned',
    'answered',
    'callback',
    'redirected',
    'transferred'
]

rows_to_order = {
    "number_of_interactions":0,
    "answered":1,
    "average_queue_time":2,
    "agent_handling_time":3,
    "average_handling_time":4,
    "abandoned":5,
    "transferred":6,
    "redirected":7,
    "callback":8
}

rows_to_rename = {
    "number_of_interactions":"Total Calls Offered",
    "answered":"% Calls Answered",
    "average_queue_time":"Average Queue Time (mins)",
    "agent_handling_time":"Total Handling Time (days)",
    "average_handling_time":"Average Handling Time (mins)",
    "abandoned":"% Calls Abandoned",
    "transferred":"% Calls Transferred",
    "redirected":"% Calls Redirected",
    "callback":"% Calls Callback"
}

In [10]:
telephony_data = report_df.groupby('monthofqueue').agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum'),
    abandoned = ('abandoned','sum'),
    answered = ('answered','sum'),
    callback = ('callback','sum'),
    redirected = ('redirected','sum'),
    transferred = ('transferred','sum'),
).reset_index()

telephony_data['queue_date'] = pd.to_datetime(telephony_data['monthofqueue'], format='%Y-%m')
telephony_data['month'] = telephony_data['queue_date'].dt.month
telephony_data['tax_year'] = telephony_data['queue_date'].apply(lambda x: x.year if x.month >= 4 else x.year-1)

for column in columns_to_cummulate:
    telephony_data[column] = telephony_data.groupby('tax_year')[column].cumsum()

telephony_data['average_handling_time'] = telephony_data['agent_handling_time'] / telephony_data['number_of_interactions']
telephony_data['average_queue_time'] = telephony_data['queue_time'] / telephony_data['number_of_interactions']


call_result_list = results_data.columns.tolist()
call_result_list = [x for x in call_result_list if x in telephony_data.columns.to_list()]
telephony_data[[f"{col}" for col in call_result_list]] = telephony_data[call_result_list].div(telephony_data["number_of_interactions"], axis=0)

telephony_data = telephony_data[telephony_data['month'] == latest_date.month-1]
telephony_data.drop(
    columns=['month','queue_date','queue_time','monthofqueue'],
    inplace=True
)

telephony_data.set_index(keys='tax_year', inplace=True)

telephony_data.rename(
    index = lambda year:f"{year} (Apr-{last_full_month_name})",
    inplace=True
)

telephony_data = telephony_data.T

pct_change = telephony_data.pct_change(axis=1)

# Format agent_handling_time (row level now, not column)
if "agent_handling_time" in telephony_data.index:
    telephony_data.loc["agent_handling_time"] = telephony_data.loc["agent_handling_time"].apply(
        lambda x: f"{int(x // 86400)}d {int((x % 86400) // 3600)}h"
    )
 
# Format average handling & queue times (also row level now)
for row in ["average_handling_time", "average_queue_time"]:
    if row in telephony_data.index:
        telephony_data.loc[row] = telephony_data.loc[row].apply(
            lambda x: f"{int(x // 60)}m {x % 60:.0f}s"
        )

telephony_data = pd.concat(
    [telephony_data, pct_change.add_suffix(r" % Change")],
    axis=1
)

telephony_data = telephony_data.replace([np.inf, -np.inf], np.nan).dropna(
    axis=1,
    how='all'
)

telephony_data.fillna('N/A', inplace=True)

telephony_data = telephony_data.reset_index().rename(
    columns={'index':'Metric'}
)
telephony_data['metric_order'] = telephony_data['Metric'].map(rows_to_order)

telephony_data.sort_values(
    by='metric_order',
    inplace=True
)

telephony_data['Metric'] = telephony_data['Metric'].map(rows_to_rename)
telephony_data.drop(
    columns='metric_order',
    inplace=True
)

mf.export_file(
    telephony_data,
    directory = path_to_matrix,
    df_name="Netcall's Telephony Matrix",
    file_type='xlsx'
)

Exporting Netcall's Telephony Matrix to XLSX...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\Netcall's Telephony Matrix.xlsx

Error exporting to XLSX: can't set attribute


In [11]:
contact_centre_data = report_df[report_df['group'] == 'Customer Services']

contact_centre_data = rmf.calculate_pct(contact_centre_data, ['group','queuesummary'], 'number_of_interactions')

contact_centre_data = contact_centre_data[contact_centre_data['monthofqueue'] == last_full_month]

Transformations being applied to the Service Data

In [12]:
service_data = report_df.groupby(['monthofqueue','month','group']).agg(
    number_of_interactions = ('number_of_interactions','sum'),
    agent_handling_time = ('agent_handling_time','sum'),
    queue_time = ('queue_time','sum'),
    abandoned = ('abandoned','sum'),
    answered = ('answered','sum'),
    callback = ('callback','sum'),
    redirected = ('redirected','sum'),
    transferred = ('transferred','sum'),
).reset_index()

service_data = rmf.calculate_pct(service_data, ['group'], 'number_of_interactions')

service_data = service_data[service_data['monthofqueue'] == last_full_month]

Final Matrix Generation

In [13]:
group_telephony_data = pd.concat([service_data, contact_centre_data])
group_telephony_data.loc[group_telephony_data['queuesummary'].notna(), 'group'] = group_telephony_data['queuesummary']
group_telephony_data['average_handling_time'] = group_telephony_data['agent_handling_time'] / group_telephony_data['number_of_interactions']
group_telephony_data['average_queue_time'] = group_telephony_data['queue_time'] / group_telephony_data['number_of_interactions']

call_result_list = results_data.columns.tolist()
call_result_list = [x for x in call_result_list if x in group_telephony_data.columns.to_list()]
group_telephony_data[[f"{col}" for col in call_result_list]] = group_telephony_data[call_result_list].div(group_telephony_data["number_of_interactions"], axis=0)
group_telephony_data.fillna(0, inplace=True)

group_telephony_data['agent_handling_time'] = group_telephony_data['agent_handling_time'].apply(
    lambda x: f"{int(x // 86400)}d {int((x % 86400) // 3600)}h"
)

for column in ['average_handling_time','average_queue_time']:
    group_telephony_data[column] = group_telephony_data[column].apply(
        lambda x: f"{int(x // 60)}m {x % 60:.0f}s"
    )

group_telephony_data['group'] = group_telephony_data['group'].replace('Customer Services','Contact Centre')

group_telephony_data['group_order'] = group_telephony_data['group'].map(groups_to_order)
group_telephony_data.sort_values(
    by='group_order',
    inplace=True
)

group_telephony_data.drop(
    columns=['monthofqueue','month','queuesummary','group_order','rolling_12m_sum', 'queue_time'],
    inplace=True
)

group_telephony_data.rename(
    columns=columns_to_rename,
    inplace=True
)

group_telephony_data = group_telephony_data[columns_to_order]

mf.display(group_telephony_data, max_rows=True)

DataFrame: group_telephony_data
Number of records: 21 | Number of fields: 12
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,Group,Total Calls,% Change Same Month Last Year,% Change 12 Month Average,% Answered,% Abandoned,% Transferred,% Callback,% Redirected,Total Handling Time (Days),Average Handling Time (Minutes),Average Queue Time (Minutes)
314,Contact Centre,25736,0.008187,-0.011152,0.587737,0.149363,0.118861,0.133820,0.010219,87d 3h,4m 53s,12m 35s
807,Accessible Transport,348,-0.689563,-0.303549,0.640805,0.074713,0.252874,0.017241,0.014368,1d 1h,4m 22s,5m 36s
808,Benefits,1606,-0.461255,-0.360926,0.491905,0.186177,0.085928,0.235990,0.000000,6d 3h,5m 32s,15m 9s
809,Complaints,410,0.084656,-0.214010,0.575610,0.102439,0.070732,0.000000,0.251220,2d 2h,7m 19s,8m 12s
810,Cost of Living,1244,-0.205112,-0.110282,0.940514,0.020900,0.033762,0.002412,0.002412,3d 14h,4m 10s,4m 9s
811,Council Tax,6322,-0.017408,0.092652,0.551882,0.228251,0.049035,0.150585,0.020247,26d 5h,5m 58s,17m 52s
812,Electoral Services,137,-0.863817,-0.523625,0.868613,0.072993,0.058394,0.000000,0.000000,0d 11h,5m 11s,6m 36s
813,Enforcement and Licensing,698,0.073846,0.073019,0.694842,0.080229,0.224928,0.000000,0.000000,1d 19h,3m 42s,5m 23s
814,Other,2143,0.485100,0.296377,0.759216,0.083061,0.143257,0.011199,0.003266,6d 11h,4m 20s,6m 58s
815,Parking,8682,0.800871,0.276484,0.543078,0.174153,0.043193,0.239576,0.000000,30d 1h,4m 59s,15m 56s


In [14]:
mf.export_file(
    group_telephony_data,
    directory = path_to_matrix,
    df_name="Netcall Latest Month's Matrix",
    file_type='xlsx'
)

Exporting Netcall Latest Month's Matrix to XLSX...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\Netcall Latest Month's Matrix.xlsx

Error exporting to XLSX: can't set attribute
